<a href="https://colab.research.google.com/github/liuk0804/2017-summer-workshop/blob/master/curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install ekphrasis
!pip install pyenchant

     |████████████████████████████████| 61kB 1.9MB/s 


In [11]:
!yum install enchant 

/bin/bash: yum: command not found


In [0]:
import ekphrasis

In [0]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer, Tokenizer

import unicodedata

In [0]:
import pandas as pd
import numpy as np
import pickle

In [0]:
import enchant
d = enchant.Dict("en_US")
import string

In [29]:
social_pipeline = [
        "EMOJI", "URL", "TAG", "EMAIL", "USER", "HASHTAG",
        "CASHTAG", "PHONE", "PERCENT", "MONEY", "DATE", "TIME",
        "ACRONYM", "CENSORED", "EMPHASIS", "NUMBER", "WORD" ]

simple_proc = TextPreProcessor(tokenizer = Tokenizer().tokenize,)

text_processor = TextPreProcessor(
    fix_text = True, # fix unicode error

    # pre-defined custom dictionary
    dicts=[emoticons, slangdict],
    
    # replace token into <tag> using Regex pattern
    normalize=['email', 'percent', 'money', 'phone', 'user',  'time', 'url', 'date', 'hashtag'],
    # turn token into (word, <tag>) tuple
    annotate= {'allcaps', 'elongated', 'repeated','emphasis', 'censored'},
    
    
    # for word segmentation 
    segmenter="twitter", 
    # for spell correction base on corpus statistics
    corrector="twitter", 
    
    unpack_contractions= True,  # Unpack contractions (can't -> can not)
    unpack_hashtags = False,
    spell_correct_elong= True,  # spell correction for elongated words

    tokenizer = SocialTokenizer(lowercase=True, pipeline = social_pipeline).tokenize,
)

Reading english - 1grams ...
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
You can't omit/backoff and unpack hashtags!
 unpack_hashtags will be set to False


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [19]:
!cat /content/drive/My\ Drive/colab_file/

cat: '/content/drive/My Drive/colab_file/': Is a directory


In [0]:
with open('/content/drive/My Drive/colab_file/emoticon_lst_all.pkl', 'rb') as f:
    emo_lst = pickle.load(f)

In [0]:
with open('/content/drive/My Drive/colab_file/slang.pkl', 'rb') as f:
    slang_lst = pickle.load(f)

In [0]:
with open('/content/drive/My Drive/colab_file/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')

In [21]:
!cat /content/drive/My\ Drive/colab_file/foo.txt

Hello Google Drive!

In [0]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [0]:
from content.drive.My\ Drive.colab_file.emoticons import emoticons

In [0]:
import sys
sys.path.append('/content/drive/My Drive/colab_file')

In [0]:
from emoticons import emoticons
from slangdict2 import slangdict

In [0]:
def en_word(word):
    if is_number(word):
        return False
    elif word in string.punctuation:
        return False
    elif word in slang_lst:
        return False
    elif word in emo_lst:
        return False
    elif '<' in word:
        return False
    elif d.check(word):
        return True
    
def not_en_word(word):
    try:

        if d.check(word):
            return False
        if is_number(word):
            return False
        elif word in string.punctuation:
            return False
        elif word in slang_lst:
            return False
        elif word in emo_lst:
            return False
        elif '<' in word:
            return False
        else:
            return True
    except ValueError:
        pass

In [30]:
!apt-get install libenchant1c2a

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,310 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv-perl amd64 1.7-5build6 [13.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libaspell15 amd64 0.60.7~20110707-4ubuntu0.1 [309 kB]
Get:3 http://a

In [0]:
def filter_OOV_token(oov_type, listOfElems):
    try:
        indexPosList = [ i-1 for i in range(len(listOfElems)) if listOfElems[i] == oov_type ]
        return [listOfElems[index] for index in indexPosList]
    except ValueError as e:
        pass


def Social_message_curation():
    msg = input("Input a sample message: ") or  "loovvvessss coofffeeeeee......"    
    print('')
    print('start processing...')
    simple_tok = simple_proc.pre_process_doc(msg)
    tokenized = ' '.join(text_processor.pre_process_doc(msg)).lower().split(' ')
    word_token = [i for i in tokenized if not (('<' in i) or (i in string.punctuation))]

    print('Raw Message Inputed:', msg)
    print('Standard NLP:', simple_tok)
    print('Aggresive/Advanced NLP:', word_token)
    print('OOV-curated NLP process:', tokenized)


    df = pd.DataFrame({'message': [msg]})
    df['tokenize'] = df.message.apply(text_processor.pre_process_doc)\
                               .apply(lambda x: ' '.join(x).lower().split(' '))
    df['simple_tokenize'] = df.message.apply(simple_proc.pre_process_doc)\
                               .apply(lambda x: ' '.join(x).lower().split(' '))
    df['en_IV'] = df.tokenize.apply(lambda x: list(filter(en_word, x)))
    df['en_IV'] = df['en_IV'].apply(lambda x: [stemmer.stem(word) for word in x])


    # df['allcap_oov'] = df.tokenize.apply(lambda x: filter_OOV_token('<allcaps>', x))
    df['slang_oov'] = df.tokenize.apply(lambda x: filter_OOV_token('<slang>', x))
    df['emoji_oov']  = df.tokenize.apply(lambda x: filter_OOV_token('<emoji>', x))
    df['elongated_oov']  = df.tokenize.apply(lambda x: filter_OOV_token('<elongated>', x))
    df['repeated_punct']  = df.tokenize.apply(lambda x: filter_OOV_token('<repeated>', x))

    df['misc_OOV'] = df.tokenize.apply(lambda x: list(filter(not_en_word, x)))

    df2 = df[['en_IV','slang_oov','emoji_oov','elongated_oov','repeated_punct','misc_OOV']].transpose()
    df2.columns = ['Captured_OOV']
    df2['OOV_Count'] = df2.Captured_OOV.apply(len)

    print(df2)

In [0]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')

In [36]:
Social_message_curation()

Input a sample message: 

start processing...
Raw Message Inputed: loovvvessss coofffeeeeee......
Standard NLP: ['loovvvessss', 'coofffeeeeee', '.', '.', '.', '.', '.', '.']
Aggresive/Advanced NLP: ['love', 'coffee']
OOV-curated NLP process: ['love', '<elongated>', 'coffee', '<elongated>', '.', '<repeated>']
                  Captured_OOV  OOV_Count
en_IV            [love, coffe]          2
slang_oov                   []          0
emoji_oov                   []          0
elongated_oov   [love, coffee]          2
repeated_punct             [.]          1
misc_OOV                    []          0
